## Guided Project: Building a Spam Filter with Naive Bayes
Naive Bayes methods are a set of supervised learning algorithms based on applying Bayes’ theorem with the “naive” assumption of independence between every pair of features. This method to classify documents, based on the words that appear within them. A common application for this type of software is in email spam filters.

### Exporing the dataset
The dataset is simple. It consists of two columns (Label, SMS)

In [20]:
import pandas as pd
df = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['Label', 'SMS'])
df.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [17]:
df['Label'].value_counts(normalize=True)

ham     0.865937
spam    0.134063
Name: Label, dtype: float64

There are 86% of ham (not spam) and 13% of spam message data.

Let's ramdomize the dataset and create train dataset (80% of the entire dataset) and test dataset (20% of the entire dataset)

In [2]:
# randomizing the entire dataset 
df_mixed = df.sample(frac=1, random_state=1)

# Calculate index for split
training_test_index = round(len(df_mixed) * 0.8)

train = df_mixed[:training_test_index].reset_index(drop=True) # 80% of the entire dataset
test = df_mixed[training_test_index:].reset_index(drop=True) # 20% of the entire dataset

print(train.shape)
print(test.shape)

(4458, 2)
(1114, 2)


In [3]:
print(train['Label'].value_counts(normalize=True))
print(test['Label'].value_counts(normalize=True))

ham     0.86541
spam    0.13459
Name: Label, dtype: float64
ham     0.868043
spam    0.131957
Name: Label, dtype: float64


The ratio of ham and spam messages are similar in train and test dataset we just created

### Data Cleaning

In [4]:
# replace all the non-word character with empty space
train['SMS'] = train['SMS'].str.replace('\W', ' ')
# lower the character
train['SMS'] = train['SMS'].str.lower()

In [5]:
train.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


In [6]:
# transform the data in the SMS column into list 
train['SMS'] = train['SMS'].str.split()
train['SMS'].head()

0                    [yep, by, the, pretty, sculpture]
1    [yes, princess, are, you, going, to, make, me,...
2                      [welp, apparently, he, retired]
3                                             [havent]
4    [i, forgot, 2, ask, ü, all, smth, there, s, a,...
Name: SMS, dtype: object

In [58]:
# extract the unique words in 'train['SMS']' data 
vocabulary = []
for sms in train['SMS']:
    for word in sms:
        vocabulary.append(word)
print(len(vocabulary)) # number of words in vocabulary list before set
vocabulary = list(set(vocabulary))
print(len(vocabulary)) # number of words in vocabulary list after set

72427
7783


There are 7783 number of unique words in train dataset messages

In [8]:
vocabulary[:10]

['grab',
 '5249',
 'aphex',
 'goodnight',
 'ultimatum',
 'velusamy',
 'babes',
 'asjesus',
 'askd',
 'priority']

Now let's create a dataframe that contains word counting information by using the vocaulbary list we just made

In [9]:
# create a dictionary that the key corresponds to unique_word and the value corresponds to 0 * length of train['SMS']
word_counts_per_sms = {unique_word: [0] * len(train['SMS']) for unique_word in vocabulary}

# loop through the train['SMS'] and increase its value by 1 if it contains the word
for index, sms in enumerate(train['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [10]:
# convert the dictionary to DataFrame
word_counts_df = pd.DataFrame(word_counts_per_sms)
word_counts_df.head()

,grab,5249,aphex,goodnight,ultimatum,velusamy,babes,asjesus,askd,priority,...,sandiago,frying,wesley,very,hee,trusting,quote,gdeve,bcums,stylish
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
# concatenate the word_counts dataframe into the train dataset
train_clean = pd.concat([train, word_counts_df], axis=1)
train_clean.head()

,Label,SMS,grab,5249,aphex,goodnight,ultimatum,velusamy,babes,asjesus,...,sandiago,frying,wesley,very,hee,trusting,quote,gdeve,bcums,stylish
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Finally, We got the desired dataset

### Creating Spam Filter - Calculate parameter
To create a spam filter, we are going to use the 'Naive Bayes' algorithm which is based on Bayes' Theorem. This algorithm assumes independence among the predictors even though in real-world it's hard to be independent. This is where its name 'Naive' comes from. Naive Bayes model is easy to build and particulary useful for very large dataset. Along with simplicity, Naive Bayes is known to outperform even highly sophiscated classification method.

In short, the Bayes theorem provides a way of calculating posterior probability with liklihood, class prior probability and predictor prior probability.

P(S|W) = (P(W|S)\*P(S))/(P(W|S)\*P(S)+P(W|H)\*P(H))

- P(S|W) : posterior probability, the probability that a message is a spam knowing that the word is in it
- P(W|S) : liklihood, the probability that the word appears in spam messages
- P(S) : class prior probability, overall probability that any given message is spam
- P(W|H) : liklihood, the probability that the word appears in ham messages
- P(H) : class prior probability, overall probability that any given message is ham

However, the disadvantage of this situation is the 'Rare word': when the word doesn't exist, both numerator and the denominator are equal to zero. The solution to this is to apply laplace smoothing constant 'alpha' to the equation.

In [38]:
# Isolating spam and ham messages first
spam_messages = train_clean[train_clean['Label'] == 'spam']
ham_messages = train_clean[train_clean['Label'] == 'ham']

# P(Spam) and P(Ham)
p_s = len(spam_messages) / len(train_clean)
p_h = len(ham_messages) / len(train_clean)

# N_Spam
n_words_per_spam_message = spam_messages['SMS'].apply(len)
n_spam = n_words_per_spam_message.sum()

# N_Ham
n_words_per_ham_message = ham_messages['SMS'].apply(len)
n_ham = n_words_per_ham_message.sum()

# N_Vocabulary
n_vocabulary = len(vocabulary)

# Laplace smoothing
alpha = 1

In [39]:
# initiate parameter for each word
parameter_ham = {unique_word: 0 for unique_word in vocabulary}
parameter_spam = {unique_word: 0 for unique_word in vocabulary}

# calculate the parameter for each word
for word in vocabulary:
    n_word_ham = ham_messages[word].sum()
    n_word_spam = spam_messages[word].sum()
    parameter_ham[word] = (n_word_ham + alpha) / (n_ham + alpha*n_vocabulary)
    parameter_spam[word] = (n_word_spam + alpha) / (n_spam + alpha*n_vocabulary)

The 'parameter_spam' means P(W|S)\*P(S) and 'parameter_ham' means P(W|H)\*P(H). 

The target probability, 
P(S|W) = (P(W|S)\*P(S)) / (P(W|S)\*P(S) + P(W|H)\*P(H)) which can be shortened as,

P(S|W) = parameter_spam / (parameter_spam + parameter_ham). 

If the 'parameter_spam' is greater than the 'parameter ham', the P(S|W) will be greater than 0.5 which indicates that the messages can be predicted as spam otherwise P(S|W) values will be less than 0.5 which indciates that the message can be predicted as ham.

### Creating Spam Filter - Algorithm

In [42]:
import re

def classify(message):
    '''
    message: a string
    '''
    
    message = re.sub('\W', ' ', message)
    message = message.lower().split()
    
    p_spam_given_message = p_s
    p_ham_given_message = p_h

    for word in message:
        if word in parameter_spam:
            p_spam_given_message *= parameter_spam[word]
        if word in parameter_ham:
            p_ham_given_message *= parameter_ham[word]
            
    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)
    
    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [59]:
test_1 = 'WINNER!! This is the secret code to unlock the money: C3421.'
test_2 = "Sounds good, Tom, then see u there"

print(classify(test_1))
print('\n')
print(classify(test_2))

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam
None


P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham
None


### Test the model

In [45]:
def classify_test_set(message):    
    '''
    message: a string
    '''
    
    message = re.sub('\W', ' ', message)
    message = message.lower().split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameter_spam:
            p_spam_given_message *= parameter_spam[word]
            
        if word in parameter_ham:
            p_ham_given_message *= parameter_ham[word]
    
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

In [46]:
test['predicted'] = test['SMS'].apply(classify_test_set)
test.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


In [60]:
correct = 0
total = test.shape[0]
incorrect_label_ham_predicted_spam = 0
incorrect_label_spam_predicted_ham = 0
    
for row in test.iterrows():
    row = row[1]
    if row['Label'] == row['predicted']:
        correct += 1
    if row['Label'] == 'ham' and row['predicted'] == 'spam':
        incorrect_label_ham_predicted_spam += 1
    if row['Label'] == 'spam' and row['predicted'] == 'ham':
        incorrect_label_spam_predicted_ham += 1
        
print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy:', correct/total)

Correct: 1100
Incorrect: 14
Accuracy: 0.9874326750448833


### Conclusion

The accuracy seems reliable which is about 98.7%. However, there are cases . In order to increase the accuracy, we have to put extra effort to critical cases where label as 'ham' but predicted as 'spam' since those cases should not exist comparing to the other cases where labeled as 'spam' but predicted as 'ham'. 

In [61]:
print('Incorrect (Label \'ham\' / Predicted = \'spam\'): ', incorrect_label_ham_predicted_spam)
print('Incorrect (Label \'spam\' / Predicted = \'ham\'): ', incorrect_label_spam_predicted_ham)

Incorrect (Label 'ham' / Predicted = 'spam'):  5
Incorrect (Label 'spam' / Predicted = 'ham'):  8
